In [1]:
import string
import pandas as pd
import numpy as np
import torch
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# nltk.download('wordnet')

In [3]:
data = 'data/WASSA23_conv_level_with_labels_train.tsv'
df = pd.read_table(data, header=0)
new_col = []
for names in df.columns:
    new_col.append(names.strip())
df.columns = new_col
dataset = df.drop(["conversation_id", "turn_id", "speaker_number", "article_id", "speaker_id", "essay_id"], axis=1, inplace=True)

X_data, y_data = df.loc[:, 'text'], df.drop('text', axis=1)
X_train, X_test, y_train , y_test = train_test_split(X_data, y_data, train_size=0.8)
#reset index of training examples
X_train, X_test = X_train.reset_index(drop=True), X_test.reset_index(drop=True)
y_train, y_test = y_train.reset_index(drop=True), y_test.reset_index(drop=True)

In [5]:
df

,text,EmotionalPolarity,Emotion,Empathy
0,I feel very sad for the people. ...,2.0000,3.0000,3.3333
1,It's terrible. Not only the people but the ani...,2.0000,4.0000,3.3333
2,I felt really sorry for the sister that now ha...,2.0000,3.6667,2.6667
3,"Yeah, it's going to be tough but i am sure she...",0.6667,3.0000,2.0000
4,"Yeah, we never know what we can do unless we a...",0.3333,2.3333,1.3333
...,...,...,...,...
8771,I'm sure that would go a long way. It's at lea...,2.0000,3.0000,3.0000
8772,"Oh exactly, it always comes down to the money....",0.0000,2.0000,2.0000
8773,And when it's the officials who are getting pa...,2.0000,2.0000,2.0000
8774,Yes corruption I'm sure is very huge. It has t...,2.0000,3.0000,3.0000


- tokenization
- remove stop word and punctuatuons, numbers
- lematization
- vectorization

In [77]:
def word_preprocessor(sentence):
    stop_words = set(stopwords.words('english'))
    punctuations = set(string.punctuation)
    lem = WordNetLemmatizer().lemmatize
    sentence = word_tokenize(sentence)
    sentence = [word for word in sentence if word not in stop_words]
    sentence = [word for word in sentence if word not in punctuations]
    sentence_str = ' '.join(sentence)
    sentence = lem(sentence_str)
    return sentence #sentence

In [78]:
X_train = X_train.apply(word_preprocessor)
X_test = X_test.apply(word_preprocessor)

#convert labels to array
X_train, X_test = np.array(X_train), np.array(X_test)
y_train, y_test = np.array(y_train[['EmotionalPolarity', 'Emotion', 'Empathy']]), np.array(y_test[['EmotionalPolarity', 'Emotion', 'Empathy']])
y_train, y_test


(array([[1.6667, 2.3333, 1.    ],
        [1.3333, 2.    , 3.    ],
        [1.3333, 2.    , 2.    ],
        ...,
        [1.    , 1.    , 1.    ],
        [2.    , 4.    , 3.3333],
        [1.6667, 3.    , 1.6667]]),
 array([[0.6667, 4.    , 2.    ],
        [1.    , 1.    , 0.6667],
        [2.    , 2.3333, 2.6667],
        ...,
        [2.    , 3.3333, 3.6667],
        [0.6667, 2.    , 1.    ],
        [1.    , 2.    , 1.3333]]))

In [93]:
vectorizer = CountVectorizer(max_features=3000, stop_words='english', lowercase=True)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [94]:
X_train_vec = X_train_vec.toarray()
X_test_vec = X_test_vec.toarray()

In [95]:
# Create a sequential model
EEPD_Model = Sequential()
EEPD_Model.add(Dense(100, activation='relu', input_dim=X_train_vec.shape[1]))
EEPD_Model.add(Dense(80, activation='relu'))
EEPD_Model.add(Dense(50, activation='relu'))
EEPD_Model.add(Dense(3, activation='softmax'))

# Compile the model
EEPD_Model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# Train the model
EEPD_Model.fit(X_train_vec, y_train, epochs=20, batch_size=100, validation_data=(X_test_vec, y_test))

# Evaluate the model
loss, accuracy = EEPD_Model.evaluate(X_test_vec, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Epoch 1/20
71/71 [==============================] - 3s 14ms/step - loss: 2.9720 - accuracy: 0.5607 - val_loss: 2.8817 - val_accuracy: 0.5700
Epoch 2/20
71/71 [==============================] - 1s 12ms/step - loss: 2.9274 - accuracy: 0.6170 - val_loss: 2.8825 - val_accuracy: 0.5609
Epoch 3/20
71/71 [==============================] - 1s 13ms/step - loss: 2.9072 - accuracy: 0.6736 - val_loss: 2.8871 - val_accuracy: 0.5478
Epoch 4/20
71/71 [==============================] - 1s 8ms/step - loss: 2.8928 - accuracy: 0.7127 - val_loss: 2.8932 - val_accuracy: 0.5444
Epoch 5/20
71/71 [==============================] - 1s 8ms/step - loss: 2.8822 - accuracy: 0.7420 - val_loss: 2.8953 - val_accuracy: 0.5473
Epoch 6/20
71/71 [==============================] - 1s 8ms/step - loss: 2.8754 - accuracy: 0.7628 - val_loss: 2.9011 - val_accuracy: 0.5194
Epoch 7/20
71/71 [==============================] - 1s 10ms/step - loss: 2.8713 - accuracy: 0.7744 - val_loss: 2.9017 - val_accuracy: 0.5239
Epoch 8/20
71/71

In [106]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((7020,), (7020, 3), (1756,), (1756, 3))